In [1]:
from enhance_ocod.inference_utils import parse_addresses_from_csv, convert_to_entity_dataframe
from enhance_ocod.address_parsing_helper_functions import (load_and_prep_OCOD_data, parsing_and_expansion_process, post_process_expanded_data)
from enhance_ocod.locate_and_classify_helper_functions import (load_postcode_district_lookup, preprocess_expandaded_ocod_data, 
                                                  add_missing_lads_ocod, load_voa_ratinglist, street_and_building_matching, substreet_matching,
                                                  counts_of_businesses_per_oa_lsoa, voa_address_match_all_data, classification_type1, classification_type2,
                                                  contract_ocod_after_classification)
from enhance_ocod.price_paid_process import load_and_process_pricepaid_data
import os
import pandas as pd


model_path = "models/address_parser_full/checkpoint-35286" #checkpoint-750"
csv_path = "data/training_data/ground_truth_test_set_labels.csv"

In [2]:
from enhance_ocod.address_parsing_helper_functions import load_cleaned_labels
temp = load_cleaned_labels("data/training_data/full_dataset_no_overlaps.json")

In [3]:
ground_truth_df = pd.read_csv(csv_path)

#I only need a small number of the columns to be able to calculate the F1 score
#Everything else just makes it confusing. 
#renaming is for consistancy
ground_truth_df = ground_truth_df.rename(
    columns = {'input:text':'property_address',
              'input:datapoint_id':'datapoint_id',
              'text':'label_text'})

ground_truth_df

,label,start,end,label_text,property_address,datapoint_id
0,street_number,0,3,161,"161, 163, 165, 167 and 169 uxbridge road, ealing",13755
1,street_number,5,8,163,"161, 163, 165, 167 and 169 uxbridge road, ealing",13755
2,street_number,10,13,165,"161, 163, 165, 167 and 169 uxbridge road, ealing",13755
3,street_number,15,18,167,"161, 163, 165, 167 and 169 uxbridge road, ealing",13755
4,street_number,23,26,169,"161, 163, 165, 167 and 169 uxbridge road, ealing",13755
...,...,...,...,...,...,...
33343,street_name,20,24,john,"118 metcalfe court, john harrison way, london ...",67996
33344,street_number,25,33,harrison,"118 metcalfe court, john harrison way, london ...",67996
33345,street_name,34,37,way,"118 metcalfe court, john harrison way, london ...",67996
33346,city,39,45,london,"118 metcalfe court, john harrison way, london ...",67996


In [6]:

results = parse_addresses_from_csv(
    df=ground_truth_df,
    model_path=str(model_path),
    target_column="property_address",
    batch_size=2048
)

Processing 33348 addresses in batches of 2048


Processing batches:   0%|          | 0/17 [00:00<?, ?batch/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0615 06:07:58.803000 19446 /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/_inductor/utils.py:1250] [1/0] Not enough SMs to use max_autotune_gemm mode
Processing batches: 100%|██████████| 17/17 [01:32<00:00,  5.45s/batch]


In [8]:
results['results']

[{'row_index': 0,
  'original_address': '161, 163, 165, 167 and 169 uxbridge road, ealing',
  'entities': [{'type': 'street_number', 'text': '161', 'start': 0, 'end': 3}],
  'parsed_components': {'street_number': '161'}},
 {'row_index': 1,
  'original_address': '161, 163, 165, 167 and 169 uxbridge road, ealing',
  'entities': [{'type': 'street_number', 'text': '161', 'start': 0, 'end': 3}],
  'parsed_components': {'street_number': '161'}},
 {'row_index': 2,
  'original_address': '161, 163, 165, 167 and 169 uxbridge road, ealing',
  'entities': [{'type': 'street_number', 'text': '161', 'start': 0, 'end': 3}],
  'parsed_components': {'street_number': '161'}},
 {'row_index': 3,
  'original_address': '161, 163, 165, 167 and 169 uxbridge road, ealing',
  'entities': [{'type': 'street_number', 'text': '161', 'start': 0, 'end': 3}],
  'parsed_components': {'street_number': '161'}},
 {'row_index': 4,
  'original_address': '161, 163, 165, 167 and 169 uxbridge road, ealing',
  'entities': [{'typ

In [15]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
import json
from collections import Counter

def check_model_labels(model_path):
    """Check what labels your trained model actually contains"""
    
    # Load the model
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    print("=== MODEL LABEL CONFIGURATION ===")
    print(f"Number of labels: {model.config.num_labels}")
    print(f"id2label mapping: {model.config.id2label}")
    print(f"label2id mapping: {model.config.label2id}")
    
    expected_labels = [
        "building_name", "street_name", "street_number", "filter_type",
        "unit_id", "unit_type", "city", "postcode"
    ]
    
    print("\n=== LABEL COMPARISON ===")
    model_labels = set(model.config.label2id.keys())
    
    # Remove BIO prefixes and 'O' for comparison
    model_entity_types = set()
    for label in model_labels:
        if label.startswith('B-') or label.startswith('I-'):
            model_entity_types.add(label[2:])  # Remove B- or I- prefix
    
    print(f"Expected labels: {expected_labels}")
    print(f"Model entity types: {sorted(model_entity_types)}")
    print(f"Missing from model: {set(expected_labels) - model_entity_types}")
    print(f"Extra in model: {model_entity_types - set(expected_labels)}")
    
    return model, tokenizer

# Check your model
model, tokenizer = check_model_labels(model_path)

=== MODEL LABEL CONFIGURATION ===
Number of labels: 17
id2label mapping: {0: 'O', 1: 'B-building_name', 2: 'I-building_name', 3: 'B-street_name', 4: 'I-street_name', 5: 'B-street_number', 6: 'I-street_number', 7: 'B-filter_type', 8: 'I-filter_type', 9: 'B-unit_id', 10: 'I-unit_id', 11: 'B-unit_type', 12: 'I-unit_type', 13: 'B-city', 14: 'I-city', 15: 'B-postcode', 16: 'I-postcode'}
label2id mapping: {'B-building_name': 1, 'B-city': 13, 'B-filter_type': 7, 'B-postcode': 15, 'B-street_name': 3, 'B-street_number': 5, 'B-unit_id': 9, 'B-unit_type': 11, 'I-building_name': 2, 'I-city': 14, 'I-filter_type': 8, 'I-postcode': 16, 'I-street_name': 4, 'I-street_number': 6, 'I-unit_id': 10, 'I-unit_type': 12, 'O': 0}

=== LABEL COMPARISON ===
Expected labels: ['building_name', 'street_name', 'street_number', 'filter_type', 'unit_id', 'unit_type', 'city', 'postcode']
Model entity types: ['building_name', 'city', 'filter_type', 'postcode', 'street_name', 'street_number', 'unit_id', 'unit_type']
Miss

In [16]:
def analyze_training_data(json_path):
    """Analyze what labels are actually in your training data"""
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print("=== TRAINING DATA ANALYSIS ===")
    print(f"Total examples: {len(data)}")
    
    # Count all labels
    label_counts = Counter()
    total_spans = 0
    
    for example in data:
        for span in example.get('spans', []):
            label_counts[span['label']] += 1
            total_spans += 1
    
    print(f"Total spans: {total_spans}")
    print("\nLabel distribution:")
    for label, count in sorted(label_counts.items()):
        percentage = (count / total_spans) * 100
        print(f"  {label}: {count} ({percentage:.1f}%)")
    
    # Check for missing expected labels
    expected_labels = [
        "building_name", "street_name", "street_number", "number_filter",
        "unit_id", "unit_type", "city", "postcode"
    ]
    
    found_labels = set(label_counts.keys())
    missing_labels = set(expected_labels) - found_labels
    
    if missing_labels:
        print(f"\n⚠️  MISSING LABELS IN TRAINING DATA: {missing_labels}")
    else:
        print(f"\n✅ All expected labels found in training data")
    
    return label_counts

# Analyze your training data
label_counts = analyze_training_data("data/training_data/training_data_dev.json")

=== TRAINING DATA ANALYSIS ===
Total examples: 2000
Total spans: 11176

Label distribution:
  building_name: 797 (7.1%)
  city: 1937 (17.3%)
  number_filter: 50 (0.4%)
  postcode: 1618 (14.5%)
  street_name: 2133 (19.1%)
  street_number: 2939 (26.3%)
  unit_id: 743 (6.6%)
  unit_type: 959 (8.6%)

✅ All expected labels found in training data


In [17]:
def debug_single_prediction(model, tokenizer, address):
    """Debug what the model predicts for a single address"""
    
    model.eval()
    device = next(model.parameters()).device
    
    # Tokenize
    inputs = tokenizer(
        address,
        return_tensors="pt",
        return_offsets_mapping=True,test
        padding=True,
        truncation=True,
        max_length=128
    )
    
    # Move to device (excluding offset_mapping)
    model_inputs = {k: v.to(device) for k, v in inputs.items() if k != 'offset_mapping'}
    
    # Predict
    with torch.no_grad():
        outputs = model(**model_inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_ids = predictions.argmax(dim=-1)
    
    # Get tokens and labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [model.config.id2label[pred.item()] for pred in predicted_ids[0]]
    offset_mapping = inputs["offset_mapping"][0]
    
    print(f"=== DEBUGGING: {address} ===")
    print("Token -> Label -> Text_Span")
    print("-" * 50)
    
    for i, (token, label, offset) in enumerate(zip(tokens, predicted_labels, offset_mapping)):
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            continue
        
        start_pos, end_pos = offset.tolist()
        if start_pos == 0 and end_pos == 0 and i > 0:
            text_span = "[SPECIAL]"
        else:
            text_span = address[start_pos:end_pos]
        
        print(f"{token:15} -> {label:15} -> '{text_span}'")

# Test with an address that should have street_name
test_address = "161, 163, 165, 167 and 169 uxbridge road, ealing"
debug_single_prediction(model, tokenizer, test_address)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (973226428.py, line 11)

In [7]:
from pre_processing_funcs import *
from tqdm import tqdm

In [31]:
with open("data/training_data/full_dataset_no_overlaps.json", 'r') as f:
    data = json.load(f)

In [37]:
data[5]

{'text': '81 hawksbill way, peterborough (pe2 8ns)',
 'labels': [{'labelId': '4',
   'label': 'street_number',
   'labellingFunctionId': 4,
   'groundTruthId': None,
   'start': 0,
   'end': 2,
   'text': '81'},
  {'labelId': '3',
   'label': 'street_name',
   'labellingFunctionId': 23,
   'groundTruthId': None,
   'start': 3,
   'end': 16,
   'text': 'hawksbill way'},
  {'labelId': '7',
   'label': 'city',
   'labellingFunctionId': 8,
   'groundTruthId': None,
   'start': 18,
   'end': 30,
   'text': 'peterborough'},
  {'labelId': '2',
   'label': 'postcode',
   'labellingFunctionId': 32,
   'groundTruthId': None,
   'start': 32,
   'end': 39,
   'text': 'pe2 8ns'}],
 'datapoint_id': 5,
 'title_number': 'CB392311'}

In [3]:
def transform_json(input_json):
    output_json = []
    
    for entry in input_json:
        transformed_entry = {
            "text": entry['text'],
            "spans": []
        }
        
        for label in entry['labels']:
            span = {
                "text": label['text'],
                "start": label['start'],
                "end": label['end'],
                "label": label['label']
            }
            transformed_entry["spans"].append(span)
        
        output_json.append(transformed_entry)
    
    return output_json

In [4]:
data2 = transform_json(data)

with open('data/training_data/converted_full_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(data2, f, ensure_ascii=False, indent=2)

In [8]:


for file in tqdm(['converted_full_dataset', 'training_data_dev', 'training_data_test']):
    
    temp = load_and_preprocess_data(f'data/training_data/{file}.json')
    with open(f'data/training_data/preprocessed_data/{file}.json', 'w', encoding='utf-8') as f:
        json.dump(temp, f, ensure_ascii=False, indent=2)

  0%|          | 0/3 [00:00<?, ?it/s]

Loaded 94088 examples
✓ All spans validated successfully


 33%|███▎      | 1/3 [00:15<00:31, 15.64s/it]

Loaded 2000 examples
Mismatch in example 2: expected 'st . james heights', got 'flat 9, st. james '
Mismatch in example 50: expected '01/06/22', got '30 - 44 '
Mismatch in example 66: expected 'queen anne 's gate', got '1 queen anne's gat'
Mismatch in example 137: expected '04/06/22', got 'flat 6, '
Mismatch in example 138: expected '07/11/22', got 'flat 4, '
Mismatch in example 215: expected '03/09/22', got '39 - 45 '
Mismatch in example 236: expected 'st katharine 's dock', got 'europe house, st kat'
Mismatch in example 254: expected '06/08/22', got '6 - 8 ve'
Mismatch in example 257: expected '08/10/22', got 'point a '
Mismatch in example 259: expected '05/07/22', got '5 - 7 ir'
Mismatch in example 311: expected '02/04/22', got 'flat 9, '
Mismatch in example 355: expected '01/08/22', got 'apartmen'
Mismatch in example 355: expected '01/08/22', got 'apartmen'
Mismatch in example 370: expected '02/04/22', got '2 - 4 we'
Mismatch in example 453: expected 'duchess of bedford 's walk', g

100%|██████████| 3/3 [00:16<00:00,  5.44s/it]

Loaded 1000 examples
Mismatch in example 21: expected 'w1 t 2dd', got '31 percy'
Mismatch in example 63: expected 'm6 6 ha', got '24 - 34'
Mismatch in example 63: expected '01/11/22', got '24 - 34 '
Mismatch in example 81: expected 'cv6 6 ha', got '6 chingf'
Mismatch in example 99: expected 'bury st . edmunds', got 'hall farm bungalo'
Mismatch in example 146: expected '04/05/22', got '4 - 5 an'
Mismatch in example 146: expected '07/08/22', got '4 - 5 an'
Mismatch in example 151: expected 'w1 t 3lr', got '25 - 33 '
Mismatch in example 216: expected 'st . helens', got '31 chadwick'
Mismatch in example 236: expected '10/12/22', got 'tollman '
Mismatch in example 263: expected 'st . helens', got '35 chadwick'
Mismatch in example 271: expected 'cr4 2 ha', got '14a grah'
Mismatch in example 281: expected 'll53 5 ha', got 'pysgod ll'
Mismatch in example 343: expected 'w1 g 6jx', got 'flat 6, '
Mismatch in example 389: expected '01/11/22', got '1 - 11 ('
Mismatch in example 396: expected '03/0

In [12]:
with open("data/training_data/preprocessed_data/training_data_dev.json", 'r') as f:
    data = json.load(f)

In [13]:
data[2]

{'text': 'flat 9, st. james heights, paradise walk, bexhill - on - sea (tn40 2lg)',
 'spans': [{'text': 'bexhill',
   'start': 42,
   'end': 49,
   'label': 'street_number'},
  {'text': 'paradise walk', 'start': 27, 'end': 40, 'label': 'street_name'},
  {'text': '9', 'start': 5, 'end': 6, 'label': 'unit_id'},
  {'text': 'bexhill - on - sea', 'start': 42, 'end': 60, 'label': 'city'},
  {'text': '-', 'start': 50, 'end': 51, 'label': 'street_number'},
  {'text': 'st . james heights',
   'start': 0,
   'end': 18,
   'label': 'building_name'},
  {'text': 'sea', 'start': 57, 'end': 60, 'label': 'street_number'},
  {'text': 'on', 'start': 52, 'end': 54, 'label': 'street_number'},
  {'text': 'st. james heights',
   'start': 8,
   'end': 25,
   'label': 'building_name'},
  {'text': 'flat', 'start': 0, 'end': 4, 'label': 'unit_type'},
  {'text': '-', 'start': 50, 'end': 51, 'label': 'street_number'},
  {'text': 'tn40 2lg', 'start': 62, 'end': 70, 'label': 'postcode'}]}

In [15]:
with open("data/training_data/training_data_dev.json", 'r') as f:
    data2 = json.load(f)

In [17]:
data2[2]

{'text': 'flat 9, st. james heights, paradise walk, bexhill-on-sea (tn40 2lg)',
 'spans': [{'text': 'bexhill',
   'start': 42,
   'end': 49,
   'label': 'street_number'},
  {'text': 'paradise walk', 'start': 27, 'end': 40, 'label': 'street_name'},
  {'text': '9', 'start': 5, 'end': 6, 'label': 'unit_id'},
  {'text': 'bexhill-on-sea', 'start': 42, 'end': 56, 'label': 'city'},
  {'text': '-', 'start': 49, 'end': 50, 'label': 'street_number'},
  {'text': 'st . james heights',
   'start': 8,
   'end': 25,
   'label': 'building_name'},
  {'text': 'sea', 'start': 53, 'end': 56, 'label': 'street_number'},
  {'text': 'on', 'start': 50, 'end': 52, 'label': 'street_number'},
  {'text': 'st. james heights',
   'start': 8,
   'end': 25,
   'label': 'building_name'},
  {'text': 'flat', 'start': 0, 'end': 4, 'label': 'unit_type'},
  {'text': '-', 'start': 52, 'end': 53, 'label': 'street_number'},
  {'text': 'tn40 2lg', 'start': 58, 'end': 66, 'label': 'postcode'}]}